https://stevesie.com/apps/airbnb-api/listings

In [1]:
import requests
import json
import time

In [2]:
url = "https://api.airbnb.com/v2/explore_tabs"
params = {
    'key':'d306zoyjsyarp7ifhu67rjxn52tv0t20',
    '_format':'for_explore_search_web',
    
    'currency':'KRW',
    'location':'서울',
    'tab_id':'home_tab',
    'items_per_grid':'306', # 최대 306 -> 수정할방법찾아야
#     'section_offset':'1',
#     'supports_for_you_v3':'true',
#     'timezone_offset':'300',
    'version':'1.3.9'
    }

In [31]:
resp = requests.get(url,params=params)

JSON 개념 정리

https://github.com/chankoo/python-for-data/blob/master/3.2%20JSON%2C%20XML.ipynb

In [32]:
res_jsn = resp.json() # .json() 메소드: response의 형태가 json 일경우 이 response를 파이썬 dict로 바로 매핑해준다

In [33]:
# with open('home_jsn_306.json', 'w') as fp: # 파일로 저장하자 -> review 크롤러에서 가져다 쓰기 위해서이다
#     json.dump(res_jsn,fp) # 파이썬 dict를 다시 json으로 변환하려면 json 모듈의 dump를 이용

__데이터 구조 개괄__
- res_jsn: {explore_tabs, metadata}
    -  __explore_tabs[0]__: { 'tab_id', 'tab_name', 'pagination_metadata', 'sections', 'experiments_metadata', 'home_tab_metadata'}
        - tab_id: home_tab
        - tab_name: homes
        - pagination_metadata: {'has_next_page', 'items_offset', 'section_offset', 'search_session_id'}
        - __sections: [0:not paginated, 1: paginated]__
            - sections[0]: ...'messages': [{'icon': 'icon-p2-aggregate-reviews',
                                           'style': 'MESSAGE_WITH_ICON',
                                       'title': 'Over 540,000 guest reviews for Seoul homes, with an average of 4.7 out of 5 stars.'}]...
                                       
            - __sections__[1]:{'backend_search_id', 'display_type', 'experiments_metadata', 'result_type', 'search_session_id', 'section_id', 'section_type_uid', 'title', 'is_paginated', 'bankai_section_id', 'refinements', __'listings'__, 'review_items', 'breadcrumbs', 'section_metadata', 'nearby_locations', 'localized_listing_count'}
                - ...
                - __listings: [#items_per_grid \* { {listing}, {pricing_quote}, {verified}, {verified_card} }]__
                    - __listing : 해당 숙소에 대한 50가지 features__
                - ...
        - experiments_metadata
        - home_tab_metadata: {'urgency_commitment', 'golden_ticket_urgency_commitment', 'listing_cards_urgency_commitment_metadata', 'listing_cards_price_line_urgency_commitment_metadata', 'messages', 'facets', 'overrides', 'search_feed_items', __'remarketing_ids'__, 'location', 'breadcrumbs', 'listings_count', 'search', 'geography', 'price_histogram', 'filters'}
            - __remarketing_ids__ : listing_id 담은 리스트 -> 숙소 id 바로뽑을 수 있다
  
    
    
    - metadata

In [5]:
res_jsn.keys() # 가장 큰 구조, 'explore_tabs'과 'metadata'로 나뉨

dict_keys(['explore_tabs', 'metadata'])

In [6]:
res_jsn['explore_tabs'][0].keys() # explore_tabs을 살펴보자

dict_keys(['tab_id', 'tab_name', 'pagination_metadata', 'sections', 'experiments_metadata', 'home_tab_metadata'])

In [7]:
res_jsn['explore_tabs'][0]['tab_id']
res_jsn['explore_tabs'][0]['tab_name'] # 웹에서 보고있는 탭이 homes 이다

'homes'

> home 탭으로 추정되는 화면
>![2018-11-04 13-38-47](https://user-images.githubusercontent.com/38183218/47960055-11210580-e037-11e8-865d-8f3fda99ed7b.png)

계속 살펴보자

In [8]:
# pagination 즉, 페이지가 넘어가는 것에 대한 메타데이터가 담겨있음
res_jsn['explore_tabs'][0]['pagination_metadata'] # items_per_grid 300개 이상 설정시 {'has_next_page': False} 가 떠버림

{'has_next_page': True,
 'items_offset': 100,
 'section_offset': 5,
 'search_session_id': '3cb445a2-32c0-420a-91df-b38e1c87d51e'}

In [9]:
# 본격적인 컨텐츠를 구성하는 sections[1] 의 구조
res_jsn['explore_tabs'][0]['sections'][1].keys()

dict_keys(['backend_search_id', 'display_type', 'experiments_metadata', 'result_type', 'search_session_id', 'section_id', 'section_type_uid', 'title', 'is_paginated', 'bankai_section_id', 'refinements', 'listings', 'review_items', 'breadcrumbs', 'section_metadata', 'nearby_locations', 'localized_listing_count'])

In [10]:
# listings에 숙소가 리스팅되어있다
res_jsn['explore_tabs'][0]['sections'][1]['listings'][1].keys()

dict_keys(['listing', 'pricing_quote', 'verified', 'verified_card'])

위 4가지 key중 listing에 대부분의 feature가 들어있음 (50개)

__listing dict의 key 설명__

- badges: ?
- bathroom_label, bathrooms: 화장실 수
- bed_label, bedroom_label, bedrooms: 침대 수, 침실 형태, 침실 수
- city: 표기 혼재 ex) '서울특별시', '마포구', 'Mapo-gu'
- guest_label: 수용인원?
- host_languages: 호스트 가능 언어
- host_thumbnail_url_small, host_thumbnail_url: 호스트 썸네일
- __id : 숙소 id__ 인듯?
- is_business_travel_ready: 비즈니스 트래블위한 숙소인지
- is_fully_refundable: 환불가능여부
- is_host_highly_rated: 호스트 신뢰가능 여부

... : 기타 features 생략

- kicker_content: 숙소형태 요약?
- lat, lng: 위도 경도
- localized_city: 군,구 수준 주소?
- localized_neighborhood: 지하철역 수준 주소?
- name: 숙소이름
- neighborhood: 근처 뭐있는지?
- person_capacity: 수용가능인원 (guest_label과 차이는뭐지??)
- picture_count,picture_url,picture_urls,picture: 숙소 사진 관련
- preview_amenities: 프리뷰에 보여지는 amenities들인듯 ex) 와이파이, 주방
- property_type_id: room_and_property_type 관련 id인듯
- __reviews_count: 리뷰수__
- room_type_category, room_type: 숙소형태 ex)Entire home/apt

... 기타

- star_rating: 숙소 별점
- tier_id: ?
- __user: 호스트 관련 정보(id, 썸네일..)__
- public_address: 군,구 수준까지 주소
- seo_reviews: ?
- amenity_ids, preview_amenity_names: 보유한 amenity들??
- reviews: ??


In [11]:
# listing의 키밸류 쌍을 뽑아보자
for k,v in res_jsn['explore_tabs'][0]['sections'][1]['listings'][10]['listing'].items():
     print("|| %30s || : %s" % (k,v)) # (|| key || : value) 형태로 출력


||                         badges || : []
||                 bathroom_label || : 1 bath
||                      bathrooms || : 1.0
||                      bed_label || : 2 beds
||                  bedroom_label || : 2 bedrooms
||                       bedrooms || : 2
||                           beds || : 2
||                           city || : Yongsan-gu
||                    guest_label || : 4 guests
||                 host_languages || : ['en', 'ko']
||       host_thumbnail_url_small || : https://a0.muscache.com/im/pictures/970eb53a-a8bd-4a04-a6dc-861846582771.jpg?aki_policy=profile_small
||             host_thumbnail_url || : https://a0.muscache.com/im/pictures/970eb53a-a8bd-4a04-a6dc-861846582771.jpg?aki_policy=profile_x_medium
||                             id || : 16683321
||       is_business_travel_ready || : False
||            is_fully_refundable || : True
||           is_host_highly_rated || : True
||                 is_new_listing || : False
||                  is_rebooka

In [12]:
res_jsn['explore_tabs'][0]['experiments_metadata'] # 뭐없다

[]

home_tab에 대한 메타데이터도 뽑아보자

결과적으로 여기의 remarketing_ids 에서 숙소 id를 바로 뽑는다

In [13]:
for k,v in res_jsn['explore_tabs'][0]['home_tab_metadata'].items():
    if k == 'filters': continue # filters 너무 많아 출력 안하겠다
    print("|| %30s || : %s" % (k,v)) # (|| key || : value) 형태로 출력

||             urgency_commitment || : {'message_type': 'empty_message', 'message': {}}
|| golden_ticket_urgency_commitment || : {'message_type': 'empty_message', 'message': {}}
|| listing_cards_urgency_commitment_metadata || : []
|| listing_cards_price_line_urgency_commitment_metadata || : []
||                       messages || : {}
||                         facets || : {}
||                      overrides || : {}
||              search_feed_items || : []
||                remarketing_ids || : [8213750, 11444903, 18956008, 12778317, 22320269, 4619104, 19777144, 17560209, 16683321, 16965764, 22133272, 18739581, 18967029, 20719576, 19634147, 20129439, 1753246, 17578320, 10044927, 21007367, 12857880, 14440359, 20763289, 15596154, 18385571, 4258317, 14069246, 19378056, 17481718, 15649684, 9710807, 8996724, 22056285, 20292369, 19576908, 18473432, 1077893, 16013768, 17660903, 14885947, 11802302, 5714483, 19433253, 10127540, 9715695, 7128212, 1634115, 982772, 24129062, 14482939, 20546115, 

In [14]:
# res_jsn['explore_tabs'][0]['home_tab_metadata']['filters'] # 필터 정보 들여다볼 필요있으면 켜보자

In [15]:
res_jsn['metadata'] # 마지막으로 res_json 전체에 대한 metadata이다 어떻게 고쳐야할지...

{'current_tab_id': 'home_tab',
 'request_uuid': '44734ba5-83ed-4b45-8575-dca541f1e5ee',
 'search_id': '44734ba5-83ed-4b45-8575-dca541f1e5ee',
 'query': '서울',
 'query_text': '서울특별시 · Homes',
 'current_refinement_paths': ['/homes'],
 'federated_search_id': '7ac2d6a3-e859-4cf1-aa51-1fe67d422f33',
 'federated_search_session_id': '1952caa0-a801-44bb-a9e7-cb7aabcaff7c',
 'suggestions': [{'title': 'All',
   'type': 'for_you',
   'search_params': {'params': [],
    'query': '서울',
    'refinement_paths': ['/for_you'],
    'refinement_path': '/for_you',
    'tab_id': 'all_tab'},
   'is_selected': False},
  {'title': 'Homes',
   'type': 'homes',
   'search_params': {'params': [],
    'query': '서울',
    'refinement_paths': ['/homes'],
    'refinement_path': '/homes',
    'tab_id': 'home_tab'},
   'is_selected': True},
  {'title': 'Experiences',
   'type': 'experiences',
   'search_params': {'params': [],
    'query': '서울',
    'refinement_paths': ['/experiences'],
    'refinement_path': '/experien

In [51]:
url = "https://api.airbnb.com/v2/explore_tabs"
params = {
    'key':'d306zoyjsyarp7ifhu67rjxn52tv0t20',
    '_format':'for_explore_search_web',
    
    'currency':'KRW',
    'location':'서울',
    'tab_id':'home_tab',
    'items_per_grid':'500', # 최대 306 -> 수정할방법찾아야
#     'section_offset':'1',
#     'supports_for_you_v3':'true',
    'timezone_offset':'300',
    'version':'1.3.9',
    'price_min':'0',
    'price_max':'99',
    }
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}


In [52]:
%%time
home = {}
except_cnt = 0
while int(params['price_max']) < 1000000:
    print(params['price_min'],params['price_max'])
    
    retry = 3 # request 예외발생시 재시도 횟수
    while retry !=0:
        try:
            resp = requests.get(url,params=params)
            res_jsn = resp.json()
            home_count = len(res_jsn['explore_tabs'][0]['sections'][1]['listings'])
            break

        except Exception as e:
            retry -= 1
            print('ERR: btw {} and {}'.format(params['price_min'],params['price_max']),end='|')
            print(e)
            time.sleep(1)
            continue
    
    if retry ==0:
        print('해당 데이터 못가져옴')
        except_cnt += 1
        continue
    if home_count > 0:
        print('해당가격대 숙소 count: ',home_count)
        for el in res_jsn['explore_tabs'][0]['sections'][1]['listings']:
            tmp_dict = el['listing']
            tmp_id = str(tmp_dict['id'])
#             if home.get(tmp_id) is not None: print('home_id {} is overlapped, be updated'.format(tmp_id))
            del tmp_dict['id']
            home[tmp_id] = tmp_dict
    
    if int(params['price_max']) < 100000: # 십만원 기준으로 그미만은 100원단위, 이상은 1000원 단위로 request
        params['price_min'] = str(int(params['price_min']) + 100) # 가격필터 변경(100원단위)
        params['price_max'] = str(int(params['price_min']) + 99)
    else:
        params['price_min'] = str(int(params['price_min']) + 1000) # 가격필터 변경(1000원단위)
        params['price_max'] = str(int(params['price_min']) + 999)

0 99
ERR: btw 0 and 99|Expecting value: line 1 column 1 (char 0)
ERR: btw 0 and 99|Expecting value: line 1 column 1 (char 0)
ERR: btw 0 and 99|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
0 99
100 199
ERR: btw 100 and 199|'explore_tabs'
200 299
300 399
ERR: btw 300 and 399|'explore_tabs'
ERR: btw 300 and 399|Expecting value: line 1 column 1 (char 0)
ERR: btw 300 and 399|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
300 399
ERR: btw 300 and 399|'explore_tabs'
ERR: btw 300 and 399|'explore_tabs'
ERR: btw 300 and 399|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
300 399
ERR: btw 300 and 399|Expecting value: line 1 column 1 (char 0)
ERR: btw 300 and 399|Expecting value: line 1 column 1 (char 0)
ERR: btw 300 and 399|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
300 399
ERR: btw 300 and 399|Expecting value: line 1 column 1 (char 0)
ERR: btw 300 and 399|Expecting value: line 1 column 1 (char 0)
ERR: btw 300 and 399|Expecting value: line 1 column 1 (char 0)
해당 

ERR: btw 5600 and 5699|'explore_tabs'
ERR: btw 5600 and 5699|'explore_tabs'
해당 데이터 못가져옴
5600 5699
ERR: btw 5600 and 5699|Expecting value: line 1 column 1 (char 0)
ERR: btw 5600 and 5699|Expecting value: line 1 column 1 (char 0)
해당가격대 숙소 count:  1
5700 5799
ERR: btw 5700 and 5799|'explore_tabs'
ERR: btw 5700 and 5799|'explore_tabs'
ERR: btw 5700 and 5799|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
5700 5799
ERR: btw 5700 and 5799|Expecting value: line 1 column 1 (char 0)
ERR: btw 5700 and 5799|Expecting value: line 1 column 1 (char 0)
ERR: btw 5700 and 5799|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
5700 5799
해당가격대 숙소 count:  1
5800 5899
ERR: btw 5800 and 5899|'explore_tabs'
ERR: btw 5800 and 5899|Expecting value: line 1 column 1 (char 0)
ERR: btw 5800 and 5899|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
5800 5899
ERR: btw 5800 and 5899|Expecting value: line 1 column 1 (char 0)
ERR: btw 5800 and 5899|Expecting value: line 1 column 1 (char 0)
해당가격대 숙소 count

ERR: btw 13100 and 13199|Expecting value: line 1 column 1 (char 0)
해당가격대 숙소 count:  111
13200 13299
해당가격대 숙소 count:  111
13300 13399
ERR: btw 13300 and 13399|Expecting value: line 1 column 1 (char 0)
ERR: btw 13300 and 13399|Expecting value: line 1 column 1 (char 0)
ERR: btw 13300 and 13399|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
13300 13399
ERR: btw 13300 and 13399|Expecting value: line 1 column 1 (char 0)
해당가격대 숙소 count:  111
13400 13499
ERR: btw 13400 and 13499|'explore_tabs'
ERR: btw 13400 and 13499|'explore_tabs'
해당가격대 숙소 count:  111
13500 13599
ERR: btw 13500 and 13599|Expecting value: line 1 column 1 (char 0)
해당가격대 숙소 count:  111
13600 13699
해당가격대 숙소 count:  111
13700 13799
ERR: btw 13700 and 13799|'explore_tabs'
ERR: btw 13700 and 13799|Expecting value: line 1 column 1 (char 0)
해당가격대 숙소 count:  111
13800 13899
ERR: btw 13800 and 13899|Expecting value: line 1 column 1 (char 0)
ERR: btw 13800 and 13899|Expecting value: line 1 column 1 (char 0)
ERR: btw 13800 and 138

해당가격대 숙소 count:  306
20200 20299
해당가격대 숙소 count:  306
20300 20399
ERR: btw 20300 and 20399|'explore_tabs'
해당가격대 숙소 count:  306
20400 20499
ERR: btw 20400 and 20499|'explore_tabs'
ERR: btw 20400 and 20499|'explore_tabs'
ERR: btw 20400 and 20499|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
20400 20499
해당가격대 숙소 count:  306
20500 20599
ERR: btw 20500 and 20599|Expecting value: line 1 column 1 (char 0)
해당가격대 숙소 count:  306
20600 20699
ERR: btw 20600 and 20699|Expecting value: line 1 column 1 (char 0)
해당가격대 숙소 count:  306
20700 20799
ERR: btw 20700 and 20799|'explore_tabs'
해당가격대 숙소 count:  306
20800 20899
해당가격대 숙소 count:  122
20900 20999
해당가격대 숙소 count:  122
21000 21099
ERR: btw 21000 and 21099|Expecting value: line 1 column 1 (char 0)
ERR: btw 21000 and 21099|Expecting value: line 1 column 1 (char 0)
ERR: btw 21000 and 21099|'explore_tabs'
해당 데이터 못가져옴
21000 21099
해당가격대 숙소 count:  122
21100 21199
ERR: btw 21100 and 21199|Expecting value: line 1 column 1 (char 0)
ERR: btw 21100 and 2

ERR: btw 25700 and 25799|Expecting value: line 1 column 1 (char 0)
ERR: btw 25700 and 25799|'explore_tabs'
해당 데이터 못가져옴
25700 25799
해당가격대 숙소 count:  139
25800 25899
ERR: btw 25800 and 25899|'explore_tabs'
해당가격대 숙소 count:  139
25900 25999
해당가격대 숙소 count:  139
26000 26099
ERR: btw 26000 and 26099|'explore_tabs'
ERR: btw 26000 and 26099|Expecting value: line 1 column 1 (char 0)
ERR: btw 26000 and 26099|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
26000 26099
ERR: btw 26000 and 26099|'explore_tabs'
ERR: btw 26000 and 26099|Expecting value: line 1 column 1 (char 0)
ERR: btw 26000 and 26099|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
26000 26099
ERR: btw 26000 and 26099|'explore_tabs'
ERR: btw 26000 and 26099|'explore_tabs'
ERR: btw 26000 and 26099|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
26000 26099
해당가격대 숙소 count:  139
26100 26199
ERR: btw 26100 and 26199|Expecting value: line 1 column 1 (char 0)
해당가격대 숙소 count:  139
26200 26299
ERR: btw 26200 and 26299|'expl

...

ERR: btw 73800 and 73899|Expecting value: line 1 column 1 (char 0)
ERR: btw 73800 and 73899|Expecting value: line 1 column 1 (char 0)
ERR: btw 73800 and 73899|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
73800 73899
해당가격대 숙소 count:  46
73900 73999
ERR: btw 73900 and 73999|'explore_tabs'
ERR: btw 73900 and 73999|Expecting value: line 1 column 1 (char 0)
ERR: btw 73900 and 73999|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
73900 73999
해당가격대 숙소 count:  46
74000 74099
ERR: btw 74000 and 74099|Expecting value: line 1 column 1 (char 0)
ERR: btw 74000 and 74099|Expecting value: line 1 column 1 (char 0)
ERR: btw 74000 and 74099|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
74000 74099
해당가격대 숙소 count:  46
74100 74199
ERR: btw 74100 and 74199|'explore_tabs'
ERR: btw 74100 and 74199|Expecting value: line 1 column 1 (char 0)
해당가격대 숙소 count:  46
74200 74299
ERR: btw 74200 and 74299|Expecting value: line 1 column 1 (char 0)
ERR: btw 74200 and 74299|'explore_tabs'
ERR: btw 7

해당가격대 숙소 count:  74
83800 83899
해당가격대 숙소 count:  74
83900 83999
해당가격대 숙소 count:  74
84000 84099
해당가격대 숙소 count:  74
84100 84199
해당가격대 숙소 count:  74
84200 84299
해당가격대 숙소 count:  74
84300 84399
해당가격대 숙소 count:  74
84400 84499
해당가격대 숙소 count:  74
84500 84599
해당가격대 숙소 count:  74
84600 84699
해당가격대 숙소 count:  74
84700 84799
해당가격대 숙소 count:  203
84800 84899
해당가격대 숙소 count:  129
84900 84999
해당가격대 숙소 count:  129
85000 85099
해당가격대 숙소 count:  129
85100 85199
해당가격대 숙소 count:  129
85200 85299
해당가격대 숙소 count:  129
85300 85399
해당가격대 숙소 count:  129
85400 85499
해당가격대 숙소 count:  129
85500 85599
해당가격대 숙소 count:  129
85600 85699
해당가격대 숙소 count:  129
85700 85799
해당가격대 숙소 count:  129
85800 85899
해당가격대 숙소 count:  158
85900 85999
해당가격대 숙소 count:  29
86000 86099
해당가격대 숙소 count:  29
86100 86199
해당가격대 숙소 count:  29
86200 86299
해당가격대 숙소 count:  29
86300 86399
해당가격대 숙소 count:  29
86400 86499
해당가격대 숙소 count:  29
86500 86599
해당가격대 숙소 count:  29
86600 86699
해당가격대 숙소 count:  29
86700 86799
해당가격대 숙소 count:  29
86800 86

해당가격대 숙소 count:  4
178000 178999
해당가격대 숙소 count:  64
179000 179999
해당가격대 숙소 count:  62
180000 180999
해당가격대 숙소 count:  64
181000 181999
해당가격대 숙소 count:  5
182000 182999
해당가격대 숙소 count:  4
183000 183999
해당가격대 숙소 count:  6
184000 184999
해당가격대 숙소 count:  13
185000 185999
해당가격대 숙소 count:  8
186000 186999
해당가격대 숙소 count:  1
187000 187999
해당가격대 숙소 count:  5
188000 188999
해당가격대 숙소 count:  4
189000 189999
해당가격대 숙소 count:  23
190000 190999
해당가격대 숙소 count:  32
191000 191999
해당가격대 숙소 count:  14
192000 192999
해당가격대 숙소 count:  3
193000 193999
해당가격대 숙소 count:  2
194000 194999
해당가격대 숙소 count:  5
195000 195999
해당가격대 숙소 count:  8
196000 196999
해당가격대 숙소 count:  8
197000 197999
해당가격대 숙소 count:  4
198000 198999
해당가격대 숙소 count:  6
199000 199999
해당가격대 숙소 count:  88
200000 200999
해당가격대 숙소 count:  90
201000 201999
해당가격대 숙소 count:  19
202000 202999
해당가격대 숙소 count:  17
203000 203999
해당가격대 숙소 count:  2
204000 204999
해당가격대 숙소 count:  1
205000 205999
해당가격대 숙소 count:  2
206000 206999
해당가격대 숙소 count:  2
207000 207999

474000 474999
475000 475999
476000 476999
477000 477999
478000 478999
479000 479999
해당가격대 숙소 count:  4
480000 480999
해당가격대 숙소 count:  4
481000 481999
482000 482999
483000 483999
484000 484999
485000 485999
486000 486999
해당가격대 숙소 count:  1
487000 487999
해당가격대 숙소 count:  1
488000 488999
해당가격대 숙소 count:  1
489000 489999
해당가격대 숙소 count:  5
490000 490999
해당가격대 숙소 count:  4
491000 491999
492000 492999
493000 493999
해당가격대 숙소 count:  1
494000 494999
해당가격대 숙소 count:  1
495000 495999
496000 496999
497000 497999
498000 498999
499000 499999
해당가격대 숙소 count:  20
500000 500999
해당가격대 숙소 count:  20
501000 501999
해당가격대 숙소 count:  20
502000 502999
503000 503999
504000 504999
해당가격대 숙소 count:  2
505000 505999
해당가격대 숙소 count:  2
506000 506999
507000 507999
508000 508999
해당가격대 숙소 count:  1
509000 509999
해당가격대 숙소 count:  1
510000 510999
511000 511999
512000 512999
513000 513999
514000 514999
515000 515999
516000 516999
517000 517999
518000 518999
해당가격대 숙소 count:  1
519000 519999
해당가격대 숙소 count:  1
520000 5209

893000 893999
894000 894999
895000 895999
896000 896999
897000 897999
898000 898999
899000 899999
해당가격대 숙소 count:  2
900000 900999
해당가격대 숙소 count:  2
901000 901999
902000 902999
903000 903999
904000 904999
905000 905999
906000 906999
907000 907999
908000 908999
909000 909999
910000 910999
911000 911999
912000 912999
913000 913999
914000 914999
915000 915999
916000 916999
917000 917999
918000 918999
919000 919999
920000 920999
921000 921999
922000 922999
923000 923999
924000 924999
925000 925999
926000 926999
927000 927999
928000 928999
929000 929999
930000 930999
931000 931999
932000 932999
933000 933999
해당가격대 숙소 count:  1
934000 934999
해당가격대 숙소 count:  1
935000 935999
936000 936999
937000 937999
938000 938999
939000 939999
ERR: btw 939000 and 939999|'explore_tabs'
940000 940999
941000 941999
942000 942999
943000 943999
944000 944999
945000 945999
946000 946999
947000 947999
948000 948999
949000 949999
950000 950999
951000 951999
952000 952999
ERR: btw 952000 and 952999|'explore_tabs'


In [53]:
len(home.keys()) 

14610

In [54]:
with open('home_jsn_14610.json','w') as fp:
    json.dump(home,fp)